In [1]:
import os, sys
import glob
import pandas as pd
import json


PROOT=os.getcwd()

if not os.path.exists("dalton"):
    os.mkdir("dalton")
from madnessToDaltony import *
madmol_files=glob.glob("*.mol")
with open(PROOT+'/molecules/frequency.json') as json_file:
        freq_json = json.loads(json_file.read())


/home/adrianhurtado/testing/madness-test-suite/madness-test-suite


In [2]:
from daltonToJson import daltonToJson
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
madmol='HCl'
xc='hf'
operator='dipole'
def basis_set_calculation(madmol,xc,operator,basis_list):
    output_files=[]
    for basis in basis_list:
        run_directory,dal_input,mol_input=dalton_polar_input(madmol,xc,operator,basis)
        d_output_file=dal_input.split('.')[0]+"_"+mol_input.split('.')[0]+'.out'
        d_out,d_error=run_dalton(run_directory,dal_input,mol_input)
        with open(run_directory+'/'+d_output_file) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
            
            
    return output_files
        
        
outfiles=basis_set_calculation(madmol,xc,operator,basis_list)

a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.018       0.018

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.144       0.144

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       1.138       1.138

@    Final HF energy:


In [33]:
import pandas
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
#madmol='HCl'
#xc='hf'
#operator='dipole'
# create an energy table

def createDaltonTables(outfiles,basis_list):
    # generate tables given name of output files and basis_list used to generate output files
    
    data={'totalEnergy':[],'nuclearRepulsionEnergy':[],'electronEnergy':[]}
    rdata={'xx':[],'yy':[],'zz':[]}
    r_dict={}
    r_dict['frequencies']=[]
    for basis,outfile in zip(basis_list,outfiles):
        e_data=json.loads(outfile)['simulation']['calculations'][0]
        r_data=json.loads(outfile)['simulation']['calculations'][1]
        for dkeys in data.keys():
            data[dkeys].append(float(e_data['calculationResults']
                                     [dkeys]['value']))
        p_data=r_data['calculationResults']['secondOrderProp']['values']
        f_data=r_data['calculationSetup']['frequencies']
        
        for p_name,p in zip(rdata.keys(),p_data):
            c_name=p_name+'-'+basis
            r_dict[c_name]=[]
            for freq,polar in zip(f_data,p):
                r_dict[c_name].append(polar)
    r_dict['frequencies']=f_data    
    rf=pd.DataFrame.from_dict(r_dict)
        
    df=pd.DataFrame.from_dict(data,orient='columns')
    return df.T,rf

energy_table,response_table=createDaltonTables(outfiles,basis_list)
energy_table.to_csv()
print(energy_table)
print(response_table)


                                 0           1           2
totalEnergy            -460.092615 -460.107599 -460.111394
nuclearRepulsionEnergy    7.057910    7.057910    7.057910
electronEnergy         -467.150526 -467.165510 -467.169305
   frequencies  xx-aug-cc-pVDZ  yy-aug-cc-pVDZ  zz-aug-cc-pVDZ  \
0         0.00       14.711954       14.711954       17.056690   
1         0.04       14.787046       14.787046       17.134372   
2         0.08       15.020271       15.020271       17.372419   

   xx-aug-cc-pVTZ  yy-aug-cc-pVTZ  zz-aug-cc-pVTZ  xx-aug-cc-pVQZ  \
0       15.805666       15.805666       17.739975       16.120439   
1       15.895418       15.895418       17.827607       16.217889   
2       16.174061       16.174061       18.096478       16.520737   

   yy-aug-cc-pVQZ  zz-aug-cc-pVQZ  
0       16.120439       17.916459  
1       16.217889       18.008340  
2       16.520737       18.290526  


In [34]:
energy_table

,0,1,2
totalEnergy,-460.092615,-460.107599,-460.111394
nuclearRepulsionEnergy,7.057910,7.057910,7.057910
electronEnergy,-467.150526,-467.165510,-467.169305


In [75]:
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
madmol='HCl'
xc='hf'
operator='dipole'
def generateDaltonDataTable(madmol,xc,operator,basis_list):
    output_files=[]
    for basis in basis_list:
        run_directory,dal_input,mol_input=dalton_polar_input(madmol,xc,operator,basis)
        d_output_file=dal_input.split('.')[0]+"_"+mol_input.split('.')[0]+'.out'
        d_out,d_error=run_dalton(run_directory,dal_input,mol_input)
        with open(run_directory+'/'+d_output_file) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
    
    if not os.path.exists("data_tables"):
        os.mkdir("data_tables")
    
    e,r=createDaltonTables(output_files,basis_list)
    e.to_csv('data_tables/'+'e_dalton_'+madmol+'_'+xc+'_'+operator+'.csv')
    r.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+operator+'.csv')
    return e,r


In [1]:
madmols=glob.glob("molecules/*.mol")
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
xc='hf'
operator='dipole'
for madmol in madmols:
    madmol=madmol.split('/')[1].split('.')[0]
    print(madmol)
    try:
        generateDaltonDataTable(madmol,xc,operator,basis_list)
    except: KeyboardInterrupt
        os.chdir(PROOT)
        
        

IndentationError: unexpected indent (3464869993.py, line 11)

In [ ]:
P2

In [70]:
basis_list=['aug-cc-pVDZ','aug-cc-pVTZ','aug-cc-pVQZ']
def DaltonOutputToTable(madmol,xc,operator,basis_list):
    run_path='dalton'
    f_p='/'.join([run_path,xc,madmol,operator])
    output_files=[]
    for basis in basis_list:
        out_path='freq_'+madmol+'-'+basis+'.out'
        full_path='/'.join([f_p,out_path])
        with open(full_path) as outfile:
            json_obj=daltonToJson()
            output_files.append(json_obj.convert(outfile))
    
    if not os.path.exists("data_tables"):
        os.mkdir("data_tables")
    
    e,r=createDaltonTables(output_files,basis_list)
    e.to_csv('data_tables/'+'e_dalton_'+madmol+'_'+xc+'_'+operator+'.csv')
    r.to_csv('data_tables/'+'r_dalton_'+madmol+'_'+xc+'_'+operator+'.csv')
    return e,r
        
for madmol in madmols:
    madmol=madmol.split('/')[1].split('.')[0]
    try:
        DaltonOutputToTable(madmol,xc,operator,basis_list)
    except: FileNotFoundError

    

a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       3.292       3.294

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :      49.053      49.110

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :     364.966     410.894

@    Final HF energy:


 CPU and wall time for SCF :       0.020       0.021

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.165       0.165

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       1.507       1.507

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted ste

 CPU and wall time for SCF :       2.515       2.515

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :      25.197      25.224

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.005       0.005

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted ste

a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.395       0.396

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       3.233       3.234

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.293       0.293

@    Final HF energy:


In [71]:

print(os.getcwd())
os.chdir(PROOT)

# get csv file if it exists already
def getDaltonResponseData(mol,xc,operator):
    dt_path='data_tables/'
    prefix='r_dalton'
    full_path=dt_path+prefix+'_'+mol+'_'+xc+'_'+operator+'.csv'
    print(full_path)
    
    return pd.read_csv(full_path ,index_col=0)

P2=getDaltonResponseData('P2','hf','dipole')
P2

/home/adrianhurtado/testing/madness-test-suite/madness-test-suite
data_tables/r_dalton_P2_hf_dipole.csv


,frequencies,xx-aug-cc-pVDZ,yy-aug-cc-pVDZ,zz-aug-cc-pVDZ,xx-aug-cc-pVTZ,yy-aug-cc-pVTZ,zz-aug-cc-pVTZ,xx-aug-cc-pVQZ,yy-aug-cc-pVQZ,zz-aug-cc-pVQZ
0,0.0,39.816286,39.816286,69.590008,41.321,41.321,70.098768,41.549977,41.549977,70.157207


In [72]:
r=getDaltonResponseData('H2O',xc,operator)
r
       

data_tables/r_dalton_H2O_hf_dipole.csv


,frequencies,xx-aug-cc-pVDZ,yy-aug-cc-pVDZ,zz-aug-cc-pVDZ,xx-aug-cc-pVTZ,yy-aug-cc-pVTZ,zz-aug-cc-pVTZ,xx-aug-cc-pVQZ,yy-aug-cc-pVQZ,zz-aug-cc-pVQZ
0,0.0,7.325093,9.043015,8.056013,7.723695,9.16998,8.38993,7.847663,9.18942,8.488083


In [76]:

basis_list=['d-aug-cc-pVDZ','d-aug-cc-pVTZ','d-aug-cc-pVQZ']
madmol='H2O'
xc='hf'
operator='dipole'
e,r=generateDaltonDataTable(madmol,xc,operator,basis_list)
e        

a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.055       0.056

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.933       0.936

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :      10.926      10.928

@    Final HF energy:


,0,1,2
totalEnergy,-76.041808,-76.060638,-76.065977
nuclearRepulsionEnergy,9.189534,9.189534,9.189534
electronEnergy,-85.231342,-85.250172,-85.255511


In [78]:
r


,frequencies,xx-d-aug-cc-pVDZ,yy-d-aug-cc-pVDZ,zz-d-aug-cc-pVDZ,xx-d-aug-cc-pVTZ,yy-d-aug-cc-pVTZ,zz-d-aug-cc-pVTZ,xx-d-aug-cc-pVQZ,yy-d-aug-cc-pVQZ,zz-d-aug-cc-pVQZ
0,0.0,7.949811,9.149492,8.466035,7.913848,9.194772,8.525102,7.904996,9.194493,8.532258


In [79]:

basis_list=['d-aug-cc-pCVDZ','d-aug-cc-pCVTZ','d-aug-cc-pCVQZ']
madmol='H2O'
xc='hf'
operator='dipole'
e,r2=generateDaltonDataTable(madmol,xc,operator,basis_list)
r2


a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       0.063       0.063

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :       1.300       1.300

@    Final HF energy:
@    Nuclear repulsion:
@    Electronic energy:
                     | End of Wave Function Section (SIRIUS) |

Linear Response calculation
a direct, restricted step, second order MCSCF program  
READING SCF
@    Wave function type
@    Number of closed shell electrons
@    Total charge of the molecule
 CPU and wall time for SCF :      17.700      17.705

@    Final HF energy:


,frequencies,xx-d-aug-cc-pCVDZ,yy-d-aug-cc-pCVDZ,zz-d-aug-cc-pCVDZ,xx-d-aug-cc-pCVTZ,yy-d-aug-cc-pCVTZ,zz-d-aug-cc-pCVTZ,xx-d-aug-cc-pCVQZ,yy-d-aug-cc-pCVQZ,zz-d-aug-cc-pCVQZ
0,0.0,7.947198,9.143873,8.462812,7.912373,9.193942,8.525274,7.904821,9.194384,8.531946


In [80]:
r

,frequencies,xx-d-aug-cc-pVDZ,yy-d-aug-cc-pVDZ,zz-d-aug-cc-pVDZ,xx-d-aug-cc-pVTZ,yy-d-aug-cc-pVTZ,zz-d-aug-cc-pVTZ,xx-d-aug-cc-pVQZ,yy-d-aug-cc-pVQZ,zz-d-aug-cc-pVQZ
0,0.0,7.949811,9.149492,8.466035,7.913848,9.194772,8.525102,7.904996,9.194493,8.532258


In [1]:
def dalton_excited_input(madmol, xc, basis,num_states):
    molname = madmol.split('.')[0]
    dalton_inp = []
    dalton_inp.append('**DALTON INPUT')
    dalton_inp.append('.RUN RESPONSE')
    dalton_inp.append('**WAVE FUNCTIONS')
    if xc == 'hf':
        dalton_inp.append('.HF')
    else:
        dalton_inp.append('.DFT')
        dalton_inp.append(xc.capitalize())
    dalton_inp.append('**PROPERTIES')
    dalton_inp.append('.EXCITA')
    dalton_inp.append('*EXCITA')
    dalton_inp.append('.NEXCIT')
    
    states=[]
    for i in range(10):
        states.append(str(num_states))
    dalton_inp.append('  '.join(states))
        

    dalton_inp.append('**END OF DALTON INPUT')
    dalton_inp = '\n'.join(dalton_inp)
    run_dir = PROOT+"/dalton/"+xc+'/'+molname+'/'+'excited_state'
    if not os.path.exists(run_dir):
        os.makedirs(run_dir)
    madmolfile = PROOT+'/molecules/'+madmol+".mol"
    mol_input = madmol_to_dalmol(madmolfile, basis)
    dal_run_file = run_dir+'/excited.dal'
    with open(dal_run_file, 'w') as file:  # Use file to refer to the file object
        file.write(dalton_inp)
    mol_file = run_dir+"/"+molname+'-'+basis.replace('*', 'S')+'.mol'
    with open(mol_file, 'w') as file:  # Use file to refer to the file object
        file.write(mol_input)
    return run_dir, dal_run_file.split('/')[-1], mol_file.split('/')[-1]

madmol='Be'
xc='hf'

num_states=freq_json[xc][madmol]['excited_state']



NameError: name 'freq_json' is not defined